In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.misc
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

from test_utils import summary, comparator
import public_tests

%matplotlib inline


In [12]:
def identity_block(X,f,filters,training=True,initializer=random_uniform):
    f1,f2,f3=filters
    X_shortcut=X
    
    X=Conv2D(filters=f1,kernel_size=1,strides=(1,1),padding='valid',kernel_initializer=initializer(seed=0))(X)
    X=BatchNormalization(axis=3)(X,training=training)
    X=Activation('relu')(X)
    X=Conv2D(filters=f2,kernel_size=f,strides=(1,1),padding='same',kernel_initializer=initializer(seed=0))(X)
    X=BatchNormalization(axis=3)(X,training=training)
    X=Activation('relu')(X)
    X=Conv2D(filters=f3,kernel_size=1,strides=(1,1),padding='valid',kernel_initializer=initializer(seed=0))(X)
    X=BatchNormalization(axis=3)(X,training=training)
    X=Add()([X_shortcut,X])
    X=Activation('relu')(X)
    return X

In [15]:
def convolutional_block(X,f,filters,s=2,training=True,initializer=glorot_uniform):
    f1,f2,f3=filters
    X_shortcut=X
    X=Conv2D(filters=f1,kernel_size=1,strides=(s,s),padding='valid',kernel_initializer=initializer(seed=0))(X)
    X=BatchNormalization(axis=3)(X,training=training)
    X=Activation('relu')(X)
    X=Conv2D(filters=f2,kernel_size=f,strides=(1,1),padding='same',kernel_initializer=initializer(seed=0))(X)
    X=BatchNormalization(axis=3)(X,training=training)
    X=Activation('relu')(X)
    X=Conv2D(filters=f3,kernel_size=1,strides=(1,1),padding='valid',kernel_initializer=initializer(seed=0))(X)
    X=BatchNormalization(axis=3)(X,training=training)
    X_shortcut=Conv2D(filters=f3,kernel_size=1,strides=(s,s),padding='valid',kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut=BatchNormalization(axis=3)(X_shortcut,training=training)
    X=Add()([X,X_shortcut])
    X=Activation('relu')(X)
    return X

In [23]:
def ResNet50(input_shape=(64,64,3),classes=6):
    X_input=Input(input_shape)
    X=ZeroPadding2D((3,3))(X_input)

    X=Conv2D(filters=64,kernel_size=(7,7),strides=(2,2),kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((3,3),strides=(2,2))(X)

    X=convolutional_block(X,f=3,filters=[64,64,256],s=1)
    X=identity_block(X,3,[64,64,256])
    X=identity_block(X,3,[64,64,256])

    X=convolutional_block(X,f=3,filters=[128,128,512],s=2)
    X=identity_block(X,3,[128,128,512])
    X=identity_block(X,3,[128,128,512])
    X=identity_block(X,3,[128,128,512])

    X=convolutional_block(X,f=3,filters=[256,256,1024],s=2)
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])

    X=convolutional_block(X,f=3,filters=[512,512,2048],s=2)
    X=identity_block(X,3,[512,512,2048])
    X=identity_block(X,3,[512,512,2048])

    X=AveragePooling2D((2,2))(X)
    X=Flatten()(X)
    X=Dense(classes,activation='softmax',kernel_initializer=glorot_uniform(seed=0))(X)
    model=Model(inputs=X_input,outputs=X)
    return model
    

    
    

In [16]:
data_augmentation = tf.keras.Sequential([
    layers.RandomCrop(64, 64),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

In [24]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding2d_3              │ (None, 70, 70, 3)         │               0 │ input_layer_3[0][0]        │
│ (ZeroPadding2D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_69 (Conv2D)            │ (None, 32, 32, 64)        │           9,472 │ zero_padding2d_3[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_67        │ (None, 32, 32, 64)        │             256 │ conv2d_69[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_62 (Activation)    │ (None, 32, 32, 64)        │               0 │ batch_normalization_67[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 15, 15, 64)        │               0 │ activation_62[0][0]        │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_70 (Conv2D)            │ (None, 15, 15, 64)        │           4,160 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_68        │ (None, 15, 15, 64)        │             256 │ conv2d_70[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_63 (Activation)    │ (None, 15, 15, 64)        │               0 │ batch_normalization_68[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_71 (Conv2D)            │ (None, 15, 15, 64)        │          36,928 │ activation_63[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_69        │ (None, 15, 15, 64)        │             256 │ conv2d_71[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_64 (Activation)    │ (None, 15, 15, 64)        │               0 │ batch_normalization_69[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_72 (Conv2D)            │ (None, 15, 15, 256)       │          16,640 │ activation_64[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_73 (Conv2D)            │ (None, 15, 15, 256)       │          16,640 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_70        │ (None, 15, 15, 256)       │           1,02

 Total params: 23,600,006 (90.03 MB)

 Trainable params: 23,546,886 (89.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

None


In [26]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [27]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [30]:
model.fit(X_train,Y_train,epochs=10,batch_size=32)

Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.9327 - loss: 0.2229
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.9456 - loss: 0.3939
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.9736 - loss: 0.0832
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9773 - loss: 0.0885
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9760 - loss: 0.0887
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.9194 - loss: 0.3099
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.9536 - loss: 0.4098
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.9462 - loss: 0.1531
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.9803 - loss: 0.0602
Epoch 10/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.9949 - loss: 0.0352


In [31]:
preds=model.evaluate(X_test,Y_test)
print('Loss= '+str(preds[0]))
print('test accuracy = '+str(preds[1]))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.8400 - loss: 0.5964
Loss= 0.48636671900749207
test accuracy = 0.8500000238418579
